In [8]:
!python -V

Python 3.9.23


In [9]:
import pandas as pd
import numpy as np

In [10]:
import pickle

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [13]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1758017519199, experiment_id='1', last_update_time=1758017519199, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [14]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [15]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [16]:
len(df_train), len(df_val)

(73908, 61921)

In [17]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [18]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)


7.758715209663881

In [21]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Phan Dang Ha")
    
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    # rmse = np.mean_squared_error(y_val, y_pred)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

12.143233939663894


In [23]:
import xgboost as xgb

In [24]:
xgb.__version__

'2.1.4'

In [25]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [26]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():

# mlflow.xgboost.autolog()  # bật tự động log
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.20472169880371677,
        'max_depth': 17,
        'min_child_weight': 1.2402611720043835,
        'objective': 'reg:linear',
        'reg_alpha': 0.28567896734700793,
        'reg_lambda': 0.004264404814393109,
        'seed': 42 
    }

    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow' )
    # print("Validation RMSE:", rmse)


/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:38:16] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

2025/09/18 07:39:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:39:57] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/09/18 07:40:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [31]:
# Đảm bảo dùng đúng scheme:
logged_model = 'runs:/3ba5503743d5442baacca1f4f526632f/models_mlflow'



loaded_model = mlflow.pyfunc.load_model(logged_model)


In [32]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: /workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-5308f029ea454816ae422b29828e5430/artifacts
  flavor: mlflow.xgboost
  run_id: 3ba5503743d5442baacca1f4f526632f

In [35]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

xgboost_model

In [36]:
# đăng ký model với code:


mlflow.register_model(
    "runs:/3ba5503743d5442baacca1f4f526632f/models_mlflow",
    "best_case_3"
)

2025/09/18 08:02:38 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/18 08:02:38 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'best_case_3'.
2025/09/18 08:02:39 WARNING mlflow.tracking._model_registry.fluent: Run with id 3ba5503743d5442baacca1f4f526632f has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-5308f029ea454816ae422b29828e5430 instead
Created version '1' of model 'best_case_3'.


<ModelVersion: aliases=[], creation_timestamp=1758182559024, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758182559024, metrics=None, model_id=None, name='best_case_3', params=None, run_id='3ba5503743d5442baacca1f4f526632f', run_link=None, source='models:/m-5308f029ea454816ae422b29828e5430', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [43]:
# Đăng ký mô hình chuẩn:

import mlflow

client = mlflow.tracking.MlflowClient()
run_id = "3ba5503743d5442baacca1f4f526632f"

print(client.get_run(run_id).info.artifact_uri)


/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/3ba5503743d5442baacca1f4f526632f/artifacts


In [42]:
# Đăng ký mô hình chuẩn:

print(client.list_artifacts(run_id, path=""))


[<FileInfo: file_size=None, is_dir=True, path='preprocessor'>]


In [41]:
# Đăng ký mô hình chuẩn:

artifact_uri = client.get_run(run_id).info.artifact_uri
model_uri = f"{artifact_uri}/models_mlflow"

mlflow.register_model(model_uri, "best_case_4")


Registered model 'best_case_4' already exists. Creating a new version of this model...
Created version '2' of model 'best_case_4'.


<ModelVersion: aliases=[], creation_timestamp=1758183178289, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1758183178289, metrics=None, model_id=None, name='best_case_4', params=None, run_id=None, run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/3ba5503743d5442baacca1f4f526632f/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [53]:
# Chuẩn không cần chỉnh, register model:

from mlflow.models.signature import infer_signature

with mlflow.start_run():
    # DMatrix cho train/valid
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    # Siêu tham số
    best_params = {
        'learning_rate': 0.20472169880371677,
        'max_depth': 17,
        'min_child_weight': 1.2402611720043835,
        'objective': 'reg:squarederror',   # 👈 dùng 'reg:squarederror' thay cho reg:linear (deprecated)
        'reg_alpha': 0.28567896734700793,
        'reg_lambda': 0.004264404814393109,
        'seed': 42 
    }

    # Log params
    mlflow.log_params(best_params)
    
    # Train model
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    # Eval
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)  # rmse
    mlflow.log_metric('rmse', rmse)

    # Log thêm preprocessing object (DictVectorizer)
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    # Tạo schema input/output để Registry hiển thị
    signature = infer_signature(X_train, booster.predict(xgb.DMatrix(X_train)))

    # Log + register model thẳng vào Model Registry
    mlflow.xgboost.log_model(
        xgb_model=booster,
        artifact_path='models_mlflow',
        registered_model_name='best_case_5',  # 👈 tên model trong Registry
        signature=signature,
        input_example=X_train[:5]             # ví dụ input
    )

# print("Validation RMSE:", rmse)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

2025/09/18 09:06:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:06:02] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
Registered model 'best_case_5' already exists. Creating a new version of this model...
Created version '2' of model 'best_case_5'.


In [58]:
## Ví dụ Code train model để tìm ra validation-rmse tốt nhất:

from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment_id = "1"  # id của experiment

# Tìm run có rmse nhỏ nhất
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.rmse ASC"],  # sắp xếp tăng dần
    max_results=1
)
best_run = runs[0]

print("Best run:", best_run.info.run_id, "RMSE:", best_run.data.metrics["rmse"])

# Đăng ký model từ run tốt nhất này
mlflow.register_model(
    f"runs:/{best_run.info.run_id}/models_mlflow",
    "best_case"
)


Best run: 60438bf0d747426ab41d77964c9ae968 RMSE: 6.307762119399252


Registered model 'best_case' already exists. Creating a new version of this model...


MlflowException: Unable to find a logged_model with artifact_path models_mlflow under run 60438bf0d747426ab41d77964c9ae968

In [57]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

experiment_id = "1"  # thay bằng id của nyc-taxi-experiment
runs = client.search_runs(
    experiment_ids=[experiment_id],
    order_by=["metrics.rmse ASC"],  # sắp xếp theo rmse tăng dần
    max_results=1
)

best_run = runs[0]
print("Best run id:", best_run.info.run_id)
print("RMSE:", best_run.data.metrics["rmse"])

# Đăng ký model từ best run
mlflow.register_model(
    f"runs:/{best_run.info.run_id}/models_mlflow",
    "best_case"
)


Best run id: 60438bf0d747426ab41d77964c9ae968
RMSE: 6.307762119399252


Registered model 'best_case' already exists. Creating a new version of this model...


MlflowException: Unable to find a logged_model with artifact_path models_mlflow under run 60438bf0d747426ab41d77964c9ae968

In [52]:
rmse = mean_squared_error(y_val, y_pred)

In [ ]:
client = mlflow.tracking.MlflowClient()
run_id = "3ba5503743d5442baacca1f4f526632f"

print(client.get_run(run_id).info.artifact_uri)

In [ ]:
print(client.list_artifacts(run_id, path=""))

In [33]:
import mlflow.pyfunc

loaded_model = mlflow.pyfunc.load_model(logged_model)
y_pred = loaded_model.predict(X_val)  # X_val là DataFrame
print(y_pred)


[14.809331  7.256571 15.71568  ... 13.564961  6.415148  8.134587]


In [34]:
# In 10 giá trị đầu tiên
print(y_pred[:10])

# Tính RMSE so với y_val
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("Validation RMSE:", rmse)


[14.809331  7.256571 15.71568  24.437902  9.105063 17.193743 11.460967
  8.388062  9.049488 18.53353 ]
Validation RMSE: 6.313589257384053


In [35]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [36]:
xgboost_model

In [37]:
dval = xgb.DMatrix(X_val)
y_pred = booster.predict(dval)
print(y_pred[:10])


[14.809331  7.256571 15.71568  24.437902  9.105063 17.193743 11.460967
  8.388062  9.049488 18.53353 ]


In [38]:
y_pred[:10]

array([14.809331,  7.256571, 15.71568 , 24.437902,  9.105063, 17.193743,
       11.460967,  8.388062,  9.049488, 18.53353 ], dtype=float32)

In [27]:
import mlflow.pyfunc

logged_model = "runs:/e2fb330f7f4b48fcbaec54bedd2b4a62/models_mlflow"
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Test predict
y_pred = loaded_model.predict(X_val)

print(X_val)


  (np.int32(0), np.int32(1182))	1.0
  (np.int32(0), np.int32(13220))	3.66
  (np.int32(1), np.int32(2301))	1.0
  (np.int32(1), np.int32(13220))	1.1
  (np.int32(2), np.int32(2317))	1.0
  (np.int32(2), np.int32(13220))	4.93
  (np.int32(3), np.int32(13220))	6.7
  (np.int32(4), np.int32(11569))	1.0
  (np.int32(4), np.int32(13220))	1.89
  (np.int32(5), np.int32(13220))	3.3
  (np.int32(6), np.int32(12248))	1.0
  (np.int32(6), np.int32(13220))	2.51
  (np.int32(7), np.int32(1142))	1.0
  (np.int32(7), np.int32(13220))	1.68
  (np.int32(8), np.int32(12834))	1.0
  (np.int32(8), np.int32(13220))	1.44
  (np.int32(9), np.int32(11918))	1.0
  (np.int32(9), np.int32(13220))	0.0
  (np.int32(10), np.int32(3782))	1.0
  (np.int32(10), np.int32(13220))	1.9
  (np.int32(11), np.int32(966))	1.0
  (np.int32(11), np.int32(13220))	1.9
  (np.int32(12), np.int32(11535))	1.0
  (np.int32(12), np.int32(13220))	1.73
  (np.int32(13), np.int32(2252))	1.0
  :	:
  (np.int32(61907), np.int32(13220))	4.09
  (np.int32(61908), n

In [33]:
model_name = "new best model_17.09"
model_version = 1  
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")


In [35]:
from mlflow import MlflowClient

client = MlflowClient()

# Lấy tất cả models trong registry
registered_models = client.search_registered_models()

for rm in registered_models:
    print(rm.name)


best model_nyc-taxi
new best model_17.09


In [36]:
import mlflow.pyfunc

# load model đầu tiên
loaded_model = mlflow.pyfunc.load_model("models:/best model_nyc-taxi/1")

# hoặc load model thứ hai
loaded_model2 = mlflow.pyfunc.load_model("models:/new best model_17.09/1")


In [40]:
import xgboost as xgb
import mlflow
import mlflow.xgboost

# Train model
dtrain = xgb.DMatrix(X_train, label=y_train)
xgb_model = xgb.train(
    params={"objective": "reg:squarederror", "max_depth": 10},
    dtrain=dtrain,
    num_boost_round=100
)

# Log + register vào MLflow Model Registry
with mlflow.start_run() as run:
    mlflow.xgboost.log_model(
        xgb_model=xgb_model,                     # 👈 truyền model
        artifact_path="models_mlflow",           # 👈 folder artifacts trong run
        registered_model_name="new best model_17.09"  # 👈 tên trong Model Registry
    )


2025/09/17 03:52:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:52:40] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/09/17 03:52:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'new best model_17.09' already exists. Creating a new version of this model...
Created version '2' of model 'new best model_17.09'.


In [41]:
import mlflow.models

signature = mlflow.models.infer_signature(X_train, y_train)

mlflow.xgboost.log_model(
    xgb_model=xgb_model,
    artifact_path="models_mlflow",
    registered_model_name="new best model_17.09",
    signature=signature,
    input_example=X_train[:5]   # ví dụ log 5 dòng đầu
)


2025/09/17 03:53:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [03:53:19] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
Registered model 'new best model_17.09' already exists. Creating a new version of this model...
Created version '3' of model 'new best model_17.09'.


In [42]:
import mlflow.pyfunc

model_name = "new best model_17.09"
model_version = 3   # version mới nhất đã tạo

# Load model từ Model Registry
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")

# Predict thử
y_pred = loaded_model.predict(X_val)

print(y_pred[:10])   # In thử 10 giá trị dự đoán đầu tiên


[16.888964   7.1836305 18.951914  24.401669   9.1852    17.163685
 13.9659195  9.368946   8.8607855 17.083324 ]
